**Table of contents**<a id='toc0_'></a>    
- [Robot Arm eye in hand estimate obj pose](#toc1_)    
- [Starting pybullet](#toc2_)    
- [Initial Setup for pybullet](#toc3_)    
- [Generating the Robot Arm](#toc4_)    
- [Generating Colored Objects](#toc5_)    
- [Function Definitions](#toc6_)    
- [Camera Settings](#toc7_)    
- [Setting the Initial Pose of the Arm](#toc8_)    
- [Estimating the Position of the Colored Object in the Camera Coordinate System](#toc9_)    
- [Estimating the Position of the Colored Object in the World Coordinate System](#toc10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Robot Arm eye in hand estimate obj pose](#toc0_)

In this notebook, we will generate a 6-axis robot arm and explain how to estimate the position of an object of a specified color using the "camera at the end of the arm".

(For a manual summarizing the functions available in pybullet, refer to [this link](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf).)

# <a id='toc2_'></a>[Starting pybullet](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[Initial Setup for pybullet](#toc0_)

In [2]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add paths to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth
time_step = 1./240.
pybullet.setTimeStep(time_step) # Set the time elapsed per step

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")

# Set the camera position and other parameters in GUI mode
camera_distance = 3.5
camera_yaw = 180.0 # deg
camera_pitch = -40 # deg
camera_target_position = [0, 0.5, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

ven = Mesa
ven = Mesa


# <a id='toc4_'></a>[Generating the Robot Arm](#toc0_)

In [3]:
# Load the robot
arm_start_pos = [0, 0, 0.0]  # Set the initial position (x, y, z)
arm_start_orientation = pybullet.getQuaternionFromEuler([0, 0, 0])  # Set the initial orientation (roll, pitch, yaw)
arm_id = pybullet.loadURDF("../urdf/simple6d_arm_with_gripper.urdf", arm_start_pos, arm_start_orientation, useFixedBase=True)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc5_'></a>[Generating Colored Objects](#toc0_)

We will generate colored objects that we want to detect with the camera.

In [4]:
# You can change this to get different results (make sure it fits within the field of view of the camera at the end of the arm) ####
color_box_pos = [-0.5, 0.2, 0.05] # Set the initial position (x, y, z) of the colored object
#################################################################################

color_box_id = pybullet.loadURDF("../urdf/simple_box.urdf", color_box_pos, pybullet.getQuaternionFromEuler([0.0, 0.0, 0.0]), globalScaling=0.1, useFixedBase=True)

# <a id='toc6_'></a>[Function Definitions](#toc0_)

In [5]:
import cv2
import numpy as np

def detect_color_obj_pose(target_rgb, rgb_img, depth_img):
    """
    Function to obtain the center position, depth, and orientation of the first detected colored object
    
    Parameters
    ----------
    target_rgb : list
        RGB of the color to be detected
    rgb_img : numpy.ndarray
        Camera image (RGB)
    depth_img : numpy.ndarray
        Camera image (Depth)

    Returns
    -------
    obj_pose : numpy.ndarray
        Position and orientation of the colored object (x, y, z, roll, pitch, yaw)
    """

    # Convert the camera image to HSV format
    hsv_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HSV)

    # Convert RGB to HSV
    target_hsv = cv2.cvtColor(np.uint8([[target_rgb]]), cv2.COLOR_RGB2HSV)[0][0]

    # Specify the range of the color to be detected
    lower = np.array([target_hsv[0]-10, 50, 50])
    upper = np.array([target_hsv[0]+10, 255, 255])

    # Extract only the specified color
    mask = cv2.inRange(hsv_img, lower, upper)

    # Extract contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get the contour with the largest area
    max_area = 0
    max_area_contour = None
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            max_area_contour = contour

    # If no contour is found
    if max_area_contour is None:
        return None
    
    # Get the center position of the contour
    M = cv2.moments(max_area_contour)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    # Get the depth at the center position of the contour
    depth = depth_img[cy, cx]

    pos = [cx, cy, depth]
    
    return pos



def Rx(theta):
    """
    Calculate the rotation matrix around the x-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Rotation matrix around the x-axis
    """
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def Ry(theta):
    """
    Calculate the rotation matrix around the y-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Rotation matrix around the y-axis
    """
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def Rz(theta):
    """
    Calculate the rotation matrix around the z-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Rotation matrix around the z-axis
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])

def Hx(theta):
    """
    Calculate the homogeneous transformation matrix around the x-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Homogeneous transformation matrix around the x-axis
    """
    return np.array([[1, 0, 0, 0],
                     [0, np.cos(theta), -np.sin(theta), 0],
                     [0, np.sin(theta), np.cos(theta), 0],
                     [0, 0, 0, 1]])

def Hy(theta):
    """
    Calculate the homogeneous transformation matrix around the y-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Homogeneous transformation matrix around the y-axis
    """
    return np.array([[np.cos(theta), 0, np.sin(theta), 0],
                     [0, 1, 0, 0],
                     [-np.sin(theta), 0, np.cos(theta), 0],
                     [0, 0, 0, 1]])

def Hz(theta):
    """
    Calculate the homogeneous transformation matrix around the z-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Homogeneous transformation matrix around the z-axis
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0, 0],
                     [np.sin(theta), np.cos(theta), 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def Hp(x, y, z):
    """
    Calculate the homogeneous transformation matrix for translation

    Parameters
    ----------
    x : float
        Translation in the x direction
    y : float
        Translation in the y direction
    z : float
        Translation in the z direction

    Returns
    -------
    numpy.ndarray
        Homogeneous transformation matrix for translation
    """
    return np.array([[1, 0, 0, x],
                     [0, 1, 0, y],
                     [0, 0, 1, z],
                     [0, 0, 0, 1]])

# <a id='toc7_'></a>[Camera Settings](#toc0_)
Define the camera settings (focal length, intrinsic parameters, etc.).

In [6]:
# Camera settings
fov = 60 # In Pybullet, specify the vertical field of view (fov)
image_width = 224 # Image width
image_height = 224 # Image height
aspect = image_width / image_height # Aspect ratio
near = 0.05 # Minimum distance of the camera
far = 5 # Maximum distance of the camera
projection_matrix = pybullet.computeProjectionMatrixFOV(fov, aspect, near, far) # Compute the projection matrix

# Calculate the focal length
fov_rad = np.deg2rad(fov)
f = (image_height / 2) / np.tan(fov_rad / 2)

# Camera intrinsic parameters
camera_matrix = np.array([[f, 0, image_width//2],
                         [0, f, image_height//2],
                         [0, 0, 1]], dtype=np.float32)

# Distortion coefficients (assuming no distortion here)
dist_coeffs = np.array([0.0, 0.0, 0.0, 0.0, 0.0], dtype=np.float32)

# <a id='toc8_'></a>[Setting the Initial Pose of the Arm](#toc0_)

In [7]:
# Indices of the robot's joints
LINK1_JOINT_IDX = 0
LINK2_JOINT_IDX = 1
LINK3_JOINT_IDX = 2
LINK4_JOINT_IDX = 3
LINK5_JOINT_IDX = 4
LINK6_JOINT_IDX = 5
CAMERA_IDX = 6
CAMERA_TARGET_IDX = 7

# Lengths of each link
ARM_ORIGIN_X_WORLD = 0.0 # x-coordinate of the arm's origin in the world coordinate system
ARM_ORIGIN_Y_WORLD = 0.0 # y-coordinate of the arm's origin in the world coordinate system
ARM_ORIGIN_Z_WORLD = 0.8 # z-coordinate of the arm's origin in the world coordinate system (length of base_link in "simple6d_arm_with_gripper.urdf")
LINK1_LENGTH = 0.3 # Length of link1 (length of link1 in "simple6d_arm_with_gripper.urdf")
LINK2_LENGTH = 0.5 # Length of link2 (length of link2 in "simple6d_arm_with_gripper.urdf")
LINK3_LENGTH = 0.5 # Length of link3 (length of link3 in "simple6d_arm_with_gripper.urdf")
LINK4_LENGTH = 0.1 # Length of link4 (length of link4 in "simple6d_arm_with_gripper.urdf")
LINK5_LENGTH = 0.1 # Length of link5 (length of link5 in "simple6d_arm_with_gripper.urdf")
LINK6_LENGTH = 0.15 # Length of link6 (length of link6 in "simple6d_arm_with_gripper.urdf")
CAMERA_Z_SIZE = 0.01 # Length of the camera in the z-direction
CAMERA_X_OFFSET = 0.08 # x-direction offset from the origin of the link6 coordinate system to the camera

# You can change this to get different results (make sure the colorBox fits within the field of view of the camera at the end of the arm) ####
# Angles of each joint
joint1_angle = -15.0
joint2_angle = -80.0
joint3_angle = -100.0
joint4_angle = 0.0
joint5_angle = 0.0
joint6_angle = 10.0
########################################################################################

# Convert the angles of each joint to radians
q1 = np.deg2rad(joint1_angle)
q2 = np.deg2rad(joint2_angle)
q3 = np.deg2rad(joint3_angle)
q4 = np.deg2rad(joint4_angle)
q5 = np.deg2rad(joint5_angle)
q6 = np.deg2rad(joint6_angle)

# Set the angles of each joint
pybullet.resetJointState(arm_id, LINK1_JOINT_IDX, q1)
pybullet.resetJointState(arm_id, LINK2_JOINT_IDX, q2)
pybullet.resetJointState(arm_id, LINK3_JOINT_IDX, q3)
pybullet.resetJointState(arm_id, LINK4_JOINT_IDX, q4)
pybullet.resetJointState(arm_id, LINK5_JOINT_IDX, q5)
pybullet.resetJointState(arm_id, LINK6_JOINT_IDX, q6)

# <a id='toc9_'></a>[Estimating the Position of the Colored Object in the Camera Coordinate System](#toc0_)

In [8]:
# Get the position of the camera
camera_link_pose = pybullet.getLinkState(arm_id, CAMERA_IDX)[0] # Position of the camera link at the end of the arm
camera_target_link_pose = pybullet.getLinkState(arm_id, CAMERA_TARGET_IDX)[0] # Position of a virtual link set just in front of the camera link
camera_link_orientation = pybullet.getEulerFromQuaternion(pybullet.getLinkState(arm_id, CAMERA_IDX)[1]) # Orientation of the camera link at the end of the arm

# Set the up vector of the camera according to the pose of the robot arm
RW1 = Rz(theta=0) @ Ry(theta=0) @ Rx(theta=0) # Rotation matrix from world coordinate system to link1 coordinate system
R12 = Rz(theta=q1) # Rotation matrix from link1 coordinate system to link2 coordinate system
R23 = Ry(theta=q2) # Rotation matrix from link2 coordinate system to link3 coordinate system
R34 = Ry(theta=q3) # Rotation matrix from link3 coordinate system to link4 coordinate system
R45 = Rz(theta=q4) # Rotation matrix from link4 coordinate system to link5 coordinate system
R56 = Ry(theta=q5) # Rotation matrix from link5 coordinate system to link6 coordinate system
R6C = Rz(theta=q6 + np.deg2rad(-90.0)) # Rotation matrix from link6 coordinate system to camera coordinate system (rotating the camera by -90 degrees makes the captured image oriented correctly)
R = RW1 @ R12 @ R23 @ R34 @ R45 @ R56 @ R6C # Rotation matrix from link1 coordinate system to camera coordinate system
camera_up_vector = np.array([0, -1, 0]) # Default up vector of the camera
rotate_camera_up_vector = R @ camera_up_vector # Up vector of the camera according to the pose of the robot arm

# Compute the view matrix of the camera
view_matrix = pybullet.computeViewMatrix(cameraEyePosition=[camera_link_pose[0], camera_link_pose[1], camera_link_pose[2]], cameraTargetPosition=[camera_target_link_pose[0], camera_target_link_pose[1], camera_target_link_pose[2]], cameraUpVector=[rotate_camera_up_vector[0], rotate_camera_up_vector[1], rotate_camera_up_vector[2]])

# Get the camera image
_, _, rgb_img, depth_img, _ = pybullet.getCameraImage(
    width=image_width,
    height=image_height,
    viewMatrix=view_matrix,
    projectionMatrix=projection_matrix,
    renderer=pybullet.ER_BULLET_HARDWARE_OPENGL
)

# Get the position of the object of the specified color
detect_color_rgb = [0, 0, 255] # Specify the color to be detected (blue in this case)
pos = detect_color_obj_pose(detect_color_rgb, rgb_img, depth_img)
pixel_x = pos[0] # x-coordinate (position on the image)
pixel_y = pos[1] # y-coordinate (position on the image)
normal_z = pos[2] # z-coordinate (depth normalized to 0-1)
z = far * near / (far - (far - near) * normal_z) # Convert depth normalized to 0-1 to distance [m]
x = (pixel_x - image_width // 2) * z / f # Convert pixelX to distance [m]
y = (pixel_y - image_height // 2) * z / f # Convert pixelY to distance [m]

# <a id='toc10_'></a>[Estimating the Position of the Colored Object in the World Coordinate System](#toc0_)

In [9]:
# Calculate the position of the camera in the world coordinate system
TW1 = Hz(theta=0) @ Hy(theta=0) @ Hx(theta=0) @ Hp(x=ARM_ORIGIN_X_WORLD, y=ARM_ORIGIN_Y_WORLD, z=ARM_ORIGIN_Z_WORLD) # Homogeneous transformation matrix from world coordinate system to link1 coordinate system
T12 = Hz(theta=q1) @ Hp(x=0, y=0, z=LINK1_LENGTH) # Transformation matrix from link1 coordinate system to link2 coordinate system
T23 = Hy(theta=q2) @ Hp(x=0, y=0, z=LINK2_LENGTH) # Transformation matrix from link2 coordinate system to link3 coordinate system
T34 = Hy(theta=q3) @ Hp(x=0, y=0, z=LINK3_LENGTH) # Transformation matrix from link3 coordinate system to link4 coordinate system
T45 = Hz(theta=q4) @ Hp(x=0, y=0, z=LINK4_LENGTH) # Transformation matrix from link4 coordinate system to link5 coordinate system
T56 = Hy(theta=q5) @ Hp(x=0, y=0, z=LINK5_LENGTH) # Transformation matrix from link5 coordinate system to link6 coordinate system
T6C = Hz(theta=q6) @ Hp(x=CAMERA_X_OFFSET, y=0, z=LINK6_LENGTH - CAMERA_Z_SIZE / 2) # Transformation matrix from link6 coordinate system to camera coordinate system
origin_pos = np.array([0, 0, 0, 1]) # Origin position
camera_pos_world = TW1 @ T12 @ T23 @ T34 @ T45 @ T56 @ T6C @ origin_pos # Position of the camera coordinate system origin in the world coordinate system
camera_pos_world = camera_pos_world[:3] # Convert the coordinates obtained by the homogeneous transformation matrix from [x, y, z, 1] to [x, y, z]
color_obj_pos_camera = np.array([x, y, z]) # Center coordinates of the "colored object" in the camera coordinate system
color_obj_pos_world = camera_pos_world + R @ color_obj_pos_camera # Position of the "colored object" in the world coordinate system

# Draw a red line on the Pybullet screen from the "camera position" to the "estimated position of the colored object" in the world coordinate system
pybullet.addUserDebugLine(camera_pos_world, color_obj_pos_world, [1, 0, 0], lineWidth=2)

# Display the "true position" and "estimated position" of the box
boxPos, _ = pybullet.getBasePositionAndOrientation(color_box_id)
pybullet.addUserDebugText(f"true box pose ({boxPos[0]:.3f}, {boxPos[1]:.3f}, {boxPos[2]:.3f})", [2.0, 0.5, 0], textColorRGB=[1, 0, 0], textSize=1.3)
pybullet.addUserDebugText(f"eye in hand, estimate box pose ({color_obj_pos_world[0]:.3f}, {color_obj_pos_world[1]:.3f}, {color_obj_pos_world[2]:.3f})", [2.0, 1.0, 0.0], textColorRGB=[1, 0, 0], textSize=1.3)

2